In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-22 16:42:22 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-22 16:42:22 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-22 16:42:22 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-22 16:42:24,820] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-22 16:42:24 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpa_xrfxfb/test.c -o /tmp/tmpa_xrfxfb/test.o
2025-07-22 16:42:24 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpa_xrfxfb/test.o -laio -o /tmp/tmpa_xrfxfb/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-22 16:42:25 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmposq64q16/test.c -o /tmp/tmposq64q16/test.o
2025-07-22 16:42:25 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmposq64q16/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmposq64q16/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-22 16:42:26 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-22 16:42:26 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-22 16:42:26 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-22 16:42:26 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-22 16:42:27 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-22 16:42:27 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-07-22 16:42:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-22 16:42:27 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-22 16:43:12 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-22 16:43:13 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>10<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [7]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-22 16:43:25 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-22 16:43:26 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-22 16:43:26 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-22 16:43:26 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-22 16:43:26 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-22 16:43:26 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-22 16:43:26 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
202

In [35]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-22 17:25:04 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [122]:
from src.selection.data import SelectionSample, get_random_sample

sample = get_random_sample(
    people_by_category=people_by_category,
    mt=mt,
    n_distractors=5,
    get_alt_obj=True,
    # category="actor",
    # obj_idx=2,
    filter_by_lm_prediction=True,
)
print(sample)
print(sample.prompt, " >> ", f'"{sample.obj}"')
sample.prediction

Ed Sheeran -> Kanye West (2): ['Greta Gerwig', 'Hillary Clinton', 'Kanye West', 'Nicholas Sparks', 'Stephen Curry', 'Phil Mickelson']
Which person from the following list has the profession in common with Ed Sheeran?
Options: Greta Gerwig, Hillary Clinton, Kanye West, Nicholas Sparks, Stephen Curry, Phil Mickelson
Ans:  >>  "Kanye West"


[PredictedToken(token=' Kanye', prob=0.74609375, logit=20.125, token_id=59817, metadata=None),
 PredictedToken(token=' The', prob=0.061279296875, logit=17.625, token_id=578, metadata=None),
 PredictedToken(token=' Nicholas', prob=0.032958984375, logit=17.0, token_id=40796, metadata=None),
 PredictedToken(token=' K', prob=0.0289306640625, logit=16.875, token_id=735, metadata=None),
 PredictedToken(token=' Ed', prob=0.017578125, logit=16.375, token_id=3279, metadata=None)]

In [123]:
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=sample.prompt,
    return_offsets_mapping=True
)

## ATTN module patterns (layerwise average of heads)

In [124]:
from src.attention import get_attention_matrices
from src.functional import interpret_logits

attn_matrices = get_attention_matrices(
    input = tokenized,
    mt = mt,
    value_weighted=True,
    # patches = []
)

interpret_logits(tokenizer=mt, logits = attn_matrices.logits)

odict_keys(['last_hidden_state', 'past_key_values', 'attentions'])
torch.Size([128256])
2025-07-22 18:15:33 src.attention DEBUG    attentions.shape=torch.Size([80, 64, 43, 43]) | values.shape=torch.Size([80, 64, 43, 128])


[PredictedToken(token=' Kanye', prob=0.765625, logit=20.25, token_id=59817, metadata=None),
 PredictedToken(token=' The', prob=0.0556640625, logit=17.625, token_id=578, metadata=None),
 PredictedToken(token=' Nicholas', prob=0.02978515625, logit=17.0, token_id=40796, metadata=None),
 PredictedToken(token=' K', prob=0.0262451171875, logit=16.875, token_id=735, metadata=None),
 PredictedToken(token=' Ed', prob=0.01806640625, logit=16.5, token_id=3279, metadata=None)]

In [125]:
attn_matrices.attention_matrices.shape

(80, 64, 43, 43)

In [126]:
from src.attention import visualize_average_attn_matrix

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt = sample.prompt,
    tokenized=tokenized,
    layer_window=range(0, mt.n_layer, 1)
)

layer=0


--------------------------------------------------------------------------------
layer=1


--------------------------------------------------------------------------------
layer=2


--------------------------------------------------------------------------------
layer=3


--------------------------------------------------------------------------------
layer=4


...

--------------------------------------------------------------------------------
layer=76


--------------------------------------------------------------------------------
layer=77


--------------------------------------------------------------------------------
layer=78


--------------------------------------------------------------------------------
layer=79


--------------------------------------------------------------------------------


## Apply (targetted) Patchscope on different layers

In [13]:
tokenized = prepare_input(
    tokenizer=mt,
    prompts=sample.prompt,
    return_offsets_mapping=True,
)
offset_mapping = tokenized.pop("offset_mapping")[0]


In [14]:
from src.tokens import find_token_range

subj_range = find_token_range(
    string=sample.prompt,
    substring=sample.subj,
    offset_mapping=offset_mapping,
    tokenizer=mt,
    occurrence=-1,
)

print(
    f'{subj_range=} | "{mt.tokenizer.decode(tokenized.input_ids[0][range(*subj_range)])}"'
)

option_ranges = []

for option in sample.options:
    option_range = find_token_range(
        string=sample.prompt,
        substring=option,
        offset_mapping=offset_mapping,
        tokenizer=mt,
        occurrence=-1,
    )
    print(
        f'{option_range=} | "{mt.tokenizer.decode(tokenized.input_ids[0][range(*option_range)])}"'
    )
    option_ranges.append(option_range)

subj_range=(13, 15) | " Trevor Noah"
option_range=(18, 20) | " Zion Williamson"
option_range=(21, 24) | " John Mulaney"
option_range=(25, 28) | " Ariana Grande"
option_range=(29, 31) | " Joe Scarborough"
option_range=(32, 36) | " Andrey Rublev"
option_range=(37, 41) | " Samuel L. Jackson"


In [15]:
from src.functional import patchscope, get_hs
from itertools import product

pred_range = (-2, 0)
token_ranges = [pred_range, subj_range] + option_ranges
token_indices = []
for rng in token_ranges:
    token_indices.extend(list(range(rng[0], rng[1])))

layers = mt.layer_names
locations = list(product(layers, token_indices))

hs = get_hs(mt=mt, input=tokenized, locations=locations, patches=[], return_dict=True)


########################################################################
context_prof = """Name of a person -> their profession
Albert Einstein -> physicist
Michael Jordan -> basketball player
Jack Ma -> entrepreneur
placeholder ->"""

patch_layers = [5]
patch_layers = [mt.layer_name_format.format(idx) for idx in patch_layers]
########################################################################

context = context_prof
context_tok = prepare_input(
    prompts=context_prof, tokenizer=mt, return_offsets_mapping=True
)
cxt_offset_mapping = context_tok.pop("offset_mapping")[0]

h_idx = (
    find_token_range(
        string=context,
        substring="placeholder",
        offset_mapping=cxt_offset_mapping,
        tokenizer=mt.tokenizer,
        occurrence=-1,
    )[1]
    - 1
)

for layer in mt.layer_names[::3]:
    print(f"\n{layer=}")
    for cur_rng in token_ranges:
        print(
            f'\n"{mt.tokenizer.decode(tokenized.input_ids[0][range(*cur_rng)])}" | {cur_rng=}'
        )
        for token_idx in range(cur_rng[0], cur_rng[1]):
            token = mt.tokenizer.decode(tokenized.input_ids[0][token_idx])
            ll_pred = patchscope(
                mt=mt,
                h=hs[(layer, token_idx)],
                context=context,
                placeholder="placeholder",
                context_tokenized=context_tok,
                placeholder_idx=h_idx,
                patch_layers=patch_layers,
                k=15,
            )
            ll_fmt = [
                f'"{pred.token}"[p={pred.prob:.2f}, l={pred.logit:.2f}]'
                for pred in ll_pred
            ]
            print(f'{token_idx=} ["{token}"] => {ll_fmt}')

    print("=" * 150)


layer='model.layers.0'

"Ans:" | cur_rng=(-2, 0)
token_idx=-2 ["Ans"] => ['"?\n"[p=0.05, l=11.19]', '" programmer"[p=0.04, l=11.00]', '"??"[p=0.03, l=10.88]', '"???"[p=0.03, l=10.75]', '" software"[p=0.03, l=10.75]', '" unknown"[p=0.03, l=10.62]', '" student"[p=0.02, l=10.44]', '" singer"[p=0.02, l=10.38]', '" entrepreneur"[p=0.02, l=10.38]', '" artist"[p=0.02, l=10.38]', '"?\n\n"[p=0.02, l=10.31]', '"?"[p=0.02, l=10.25]', '" politician"[p=0.01, l=10.00]', '" actor"[p=0.01, l=10.00]', '" ("[p=0.01, l=9.94]']
token_idx=-1 [":"] => ['"?\n"[p=0.22, l=13.75]', '"?\n\n"[p=0.15, l=13.38]', '"??"[p=0.09, l=12.81]', '"?"[p=0.08, l=12.75]', '"???"[p=0.06, l=12.50]', '" \n"[p=0.02, l=11.56]', '" lawyer"[p=0.02, l=11.50]', '" unknown"[p=0.01, l=11.00]', '" artist"[p=0.01, l=10.75]', '" politician"[p=0.01, l=10.62]', '" software"[p=0.01, l=10.56]', '" programmer"[p=0.01, l=10.50]', '" ("[p=0.01, l=10.25]', '" "[p=0.01, l=10.25]', '" \n\n"[p=0.01, l=10.25]']

" Trevor Noah" | cur_rng=(13, 15)
toke

## Individual Heads

In [127]:
from src.attention import visualize_attn_matrix

###############################################################
N_KV_GROUPS = 1  # individual heads
N_HEADS = mt.config.num_attention_heads
LAYER_RANGE = range(mt.n_layer)
###############################################################

group_id_to_head_indices = {
    i: [i * N_KV_GROUPS + j for j in range(N_KV_GROUPS)]
    for i in range(N_HEADS // N_KV_GROUPS)
}
attn_info = get_attention_matrices(
    input=tokenized,
    mt=mt,
    # value_weighted=True,
    patches=[],
)
attn_info.attention_matrices.shape

odict_keys(['last_hidden_state', 'past_key_values', 'attentions'])
torch.Size([128256])


(80, 64, 43, 43)

In [128]:
layer_idx = 35

for group_id in group_id_to_head_indices:
    frm = group_id_to_head_indices[group_id][0]
    to = group_id_to_head_indices[group_id][-1]

    attn_matrix = torch.Tensor(attn_info.attention_matrices[layer_idx, frm:to + 1]).mean(dim=0).squeeze()
    print(
        f"Layer {layer_idx}, Group {group_id} (Heads {frm}-{to}) | Shape: {attn_matrix.shape}"
    )

    visualize_attn_matrix(
        attn_matrix=attn_matrix,  # layer 30, head 5
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer 35, Group 0 (Heads 0-0) | Shape: torch.Size([43, 43])


Layer 35, Group 1 (Heads 1-1) | Shape: torch.Size([43, 43])


Layer 35, Group 2 (Heads 2-2) | Shape: torch.Size([43, 43])


Layer 35, Group 3 (Heads 3-3) | Shape: torch.Size([43, 43])


Layer 35, Group 4 (Heads 4-4) | Shape: torch.Size([43, 43])


...

Layer 35, Group 59 (Heads 59-59) | Shape: torch.Size([43, 43])


Layer 35, Group 60 (Heads 60-60) | Shape: torch.Size([43, 43])


Layer 35, Group 61 (Heads 61-61) | Shape: torch.Size([43, 43])


Layer 35, Group 62 (Heads 62-62) | Shape: torch.Size([43, 43])


Layer 35, Group 63 (Heads 63-63) | Shape: torch.Size([43, 43])


In [172]:
from src.functional import get_hs
from src.functional import logit_lens, patchscope, get_module_nnsight
########################################################################
context_prof = """Name of a person -> their profession
Albert Einstein -> physicist
Michael Jordan -> basketball player
Jack Ma -> entrepreneur
placeholder ->"""

patch_layers = [5]
patch_layers = [mt.layer_name_format.format(idx) for idx in patch_layers]
########################################################################

layer_idx, head_idx = 35, 50

visualize_attn_matrix(
    attn_matrix=attn_info.attention_matrices[layer_idx, head_idx],
    tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
    q_index=-1,
    start_from=1,
)

# hs = get_hs(
#     mt=mt,
#     input=tokenized,
#     locations=[(layer_name, -1) for layer_name in mt.layer_names],
#     patches=[],
#     return_dict=True,
# )
with mt.trace(tokenized) as trace:
    res_prev = get_module_nnsight(mt, mt.layer_name_format.format(layer_idx-1))
    head_inp = res_prev.output[0].save()

print(f"{head_inp.shape=}")

ll_pred = logit_lens(
    mt = mt,
    h = head_inp[0, -1, :],
)
print(f'll_pred={[str(pred) for pred in ll_pred]}')

patch_pred = patchscope(
    mt=mt,
    h=head_inp[0, -1, :],
    context=context_prof,
    placeholder="placeholder",
    patch_layers=patch_layers,
    k=15,
)
print(f'patch_pred={[str(pred) for pred in patch_pred]}')

head_inp.shape=torch.Size([1, 43, 8192])
ll_pred=['"adel"[15503] (p=0.005, logit=6.906)', '"ais"[2852] (p=0.003, logit=6.625)', '"u"[84] (p=0.003, logit=6.625)', '"uss"[1892] (p=0.003, logit=6.625)', '" none"[7000] (p=0.003, logit=6.594)']
patch_pred=['"?\n"[18072] (p=0.154, logit=12.125)', '"?\n\n"[24688] (p=0.078, logit=11.438)', '"?"[949] (p=0.044, logit=10.875)', '" \n"[720] (p=0.042, logit=10.812)', '" ("[320] (p=0.027, logit=10.375)', '" ->"[1492] (p=0.021, logit=10.125)', '" "[220] (p=0.021, logit=10.125)', '"??"[9602] (p=0.015, logit=9.812)', '" lawyer"[15779] (p=0.013, logit=9.688)', '" unknown"[9987] (p=0.013, logit=9.625)', '" profession"[4913] (p=0.010, logit=9.438)', '"\n\n"[271] (p=0.010, logit=9.375)', '" artist"[10255] (p=0.010, logit=9.375)', '" software"[3241] (p=0.009, logit=9.312)', '" \n\n"[4815] (p=0.008, logit=9.188)']


In [179]:
import baukit
import types
from src.hooking.llama_attention import LlamaAttentionPatcher
from src.functional import predict_next_token

mt.reset_forward()
attn_block = baukit.get_module(
    mt._model,
    mt.attn_module_name_format.format(layer_idx),
)

patched_attn_matrices = {}
patched_head_contributions={}
attn_block.forward = types.MethodType(
    LlamaAttentionPatcher(
        block_name = mt.attn_module_name_format.format(layer_idx),
        save_attn_for=list(range(mt.config.num_attention_heads)),
        store_attn_matrices = patched_attn_matrices,
        store_head_contributions = patched_head_contributions,
    ),
    attn_block
)

patched_pred = predict_next_token(
    mt = mt,
    inputs = tokenized,
)

mt.reset_forward()

patched_pred

2025-07-22 18:55:53 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.35.self_attn
2025-07-22 18:55:53 src.hooking.llama_attention DEBUG    hidden_shape=(1, 43, -1, 128) | input_shape=torch.Size([1, 43]) | torch.Size([1, 43, 8192])
2025-07-22 18:55:54 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 43, 128]) | key_states.size()=torch.Size([1, 8, 43, 128]) | value_states.size()=torch.Size([1, 8, 43, 128])
o_proj_weight_split.size()=torch.Size([8192, 64, 128])
attn_output.size()=torch.Size([1, 43, 64, 128])


[[PredictedToken(token=' Kanye', prob=0.74609375, logit=20.125, token_id=59817, metadata=None),
  PredictedToken(token=' The', prob=0.061279296875, logit=17.625, token_id=578, metadata=None),
  PredictedToken(token=' Nicholas', prob=0.032958984375, logit=17.0, token_id=40796, metadata=None),
  PredictedToken(token=' K', prob=0.029052734375, logit=16.875, token_id=735, metadata=None),
  PredictedToken(token=' Ed', prob=0.017578125, logit=16.375, token_id=3279, metadata=None)]]

In [174]:
from src.functional import logit_lens
print(f"{patched_attn_matrices[head_idx].shape=}")
print(f"{patched_head_contributions[head_idx].shape=}")

visualize_attn_matrix(
    attn_matrix=patched_attn_matrices[head_idx].squeeze(),
    tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
    q_index=-1,
    start_from=1,
)

logit_lens(
    mt = mt,
    h = patched_head_contributions[head_idx][:, -1],
)

patched_attn_matrices[head_idx].shape=torch.Size([1, 43, 43])
patched_head_contributions[head_idx].shape=torch.Size([1, 43, 8192])


[PredictedToken(token=' breat', prob=0.00958251953125, logit=8.75, token_id=28855, metadata=None),
 PredictedToken(token=' continu', prob=0.006561279296875, logit=8.375, token_id=16513, metadata=None),
 PredictedToken(token=' Deng', prob=0.004364013671875, logit=7.96875, token_id=92829, metadata=None),
 PredictedToken(token=' Daily', prob=0.004119873046875, logit=7.90625, token_id=13690, metadata=None),
 PredictedToken(token=' continuation', prob=0.0038604736328125, logit=7.84375, token_id=42271, metadata=None)]

In [175]:
import baukit
attn_module = baukit.get_module(mt._model, mt.attn_module_name_format.format(layer_idx))
print(f"{attn_module.head_dim=} | {attn_module.num_key_value_groups=}")
attn_module

attn_module.head_dim=128 | attn_module.num_key_value_groups=8


LlamaAttention(
  (q_proj): Linear(in_features=8192, out_features=8192, bias=False)
  (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
  (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
  (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
)

In [176]:
from src.hooking.llama_attention import repeat_kv

hidden_shape = (*tokenized.input_ids.shape, -1, attn_module.head_dim)

query_states = attn_module.q_proj(head_inp).view(hidden_shape).transpose(1, 2)
key_states = attn_module.k_proj(head_inp).view(hidden_shape).transpose(1, 2)
value_states = attn_module.v_proj(head_inp).view(hidden_shape).transpose(1, 2)
print(f"{query_states.shape=}, {key_states.shape=}, {value_states.shape=}")

query_states.shape=torch.Size([1, 64, 43, 128]), key_states.shape=torch.Size([1, 8, 43, 128]), value_states.shape=torch.Size([1, 8, 43, 128])


In [186]:
head_query_out = query_states[:, head_idx, :, :]
head_query_out.shape

torch.Size([1, 43, 128])

In [219]:
q_proj_weight_split = attn_module.q_proj.weight.view(
    N_HEADS, attn_module.head_dim, attn_module.q_proj.in_features
)

print(f"{q_proj_weight_split.shape=}")
head_q_proj_weight = q_proj_weight_split[head_idx]
print(f"{head_q_proj_weight.shape=}")

head_query_out_2 = head_inp @ head_q_proj_weight.T
print(f"{head_query_out_2.shape=}")

q_proj_weight_split.shape=torch.Size([64, 128, 8192])
head_q_proj_weight.shape=torch.Size([128, 8192])
head_query_out_2.shape=torch.Size([1, 43, 128])


In [245]:
print(torch.allclose(head_query_out, head_query_out_2, atol=1e-1))
(head_query_out_2 - head_query_out)

False


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:3',
       dtype=torch.bfloat16, grad_fn=<SubBackward0>)

In [246]:
head_q_proj_weight.shape

torch.Size([128, 8192])

In [247]:
head_inp.shape

torch.Size([1, 43, 8192])

In [251]:
with torch.no_grad():
    head_q_proj = head_q_proj_weight.to(torch.float32)
    proj_matrix = head_q_proj.T @ torch.linalg.pinv(head_q_proj @ head_q_proj.T) @ head_q_proj
    head_query_read = head_inp @ proj_matrix.to(mt.dtype).T

ll_pred = logit_lens(
    mt = mt,
    h = head_query_read[:, -1].squeeze(),
    k=15
)
print(f'll_pred={[str(pred) for pred in ll_pred]}')

patch_pred = patchscope(
    mt=mt,
    h=head_query_read[:, -1].squeeze(),
    context=context_prof,
    placeholder="placeholder",
    patch_layers=patch_layers,
    k=15,
)
print(f'patch_pred={[str(pred) for pred in patch_pred]}')

proj_matrix.shape

ll_pred=['" Gow"[93571] (p=0.037, logit=9.562)', '" stalk"[55972] (p=0.003, logit=7.125)', '" Bag"[20287] (p=0.003, logit=6.906)', '"groupon"[71014] (p=0.002, logit=6.844)', '"alaxy"[109956] (p=0.002, logit=6.812)', '" bip"[29978] (p=0.002, logit=6.781)', '" ong"[127721] (p=0.002, logit=6.656)', '"ंर"[123418] (p=0.002, logit=6.531)', '" bag"[9145] (p=0.002, logit=6.531)', '" fre"[3541] (p=0.002, logit=6.469)', '" Shepherd"[59646] (p=0.002, logit=6.375)', '" Collector"[59648] (p=0.002, logit=6.375)', '"/oauth"[61803] (p=0.001, logit=6.312)', '" Trip"[27852] (p=0.001, logit=6.312)', '"kili"[118462] (p=0.001, logit=6.312)']
patch_pred=['" invalid"[8482] (p=0.070, logit=10.875)', '" unknown"[9987] (p=0.058, logit=10.688)', '"??"[9602] (p=0.055, logit=10.625)', '"?\n"[18072] (p=0.048, logit=10.500)', '"???"[52417] (p=0.045, logit=10.438)', '"?\n\n"[24688] (p=0.033, logit=10.125)', '" error"[1493] (p=0.031, logit=10.062)', '"?"[949] (p=0.028, logit=9.938)', '" not"[539] (p=0.021, logit=9.688

torch.Size([8192, 8192])

In [252]:
head_inp.shape, proj_matrix.shape

(torch.Size([1, 43, 8192]), torch.Size([8192, 8192]))